## estrazione features da reti pretrainate

In [1]:
import numpy as np
import collections
import SimpleITK as sitk
from scipy.ndimage import zoom
import nrrd
import os,sys
import pandas as pd
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import Model


In [14]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input

model = VGG19(weights='imagenet', include_top=False)

pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

# funzione per creazione maschera
def maskcroppingbox(images_array):
    images_array_2 = np.argwhere(images_array)
    
    (zstart, ystart, xstart), (zstop, ystop, xstop) = images_array_2.min(axis=0), images_array_2.max(axis=0) + 1
    return (zstart, ystart, xstart), (zstop, ystop, xstop)
        
def featureextraction(image_array,mask_array):
    # ridimensionamento
    (zstart, ystart, xstart), (zstop, ystop, xstop) = maskcroppingbox(mask_array)
    roi_images = image_array[zstart-1:zstop+1,ystart:ystop,xstart:xstop].transpose((2,1,0))
    roi_images1 = zoom(roi_images, zoom=[224/roi_images.shape[0], 224/roi_images.shape[1],1], order=3)
    roi_images2 = np.array(roi_images1,dtype=float)    

    # Trova la slice più grossa (basato sui pixel attivi della maschera)
    slice_sums = np.sum(mask_array[zstart-1:zstop+1, ystart:ystop, xstart:xstop], axis=(1, 2))  # Somma dei pixel per slice
    largest_slice_index = np.argmax(slice_sums)  # Indice della slice con il massimo numero di pixel attivi

    # Estrai solo la slice più grossa
    largest_slice_image = roi_images2[:, :, largest_slice_index]

    print(largest_slice_image.shape)

    # preprocessing per resnet
    x = image.img_to_array(largest_slice_image)
    print(x.shape)
    x = np.repeat(x, 3, axis=-1)
    x = np.expand_dims(x, axis=0)
    print(x.shape)
    x = preprocess_input(x)
    #x = np.transpose(x, (3, 1, 2, 0))
    

    # estrazione features principali come feature map
    base_model_pool_features = model.predict(x)

    feature_map = base_model_pool_features[0]

    #print(feature_map)

    feature_map = feature_map.transpose((2,1,0))
    features = np.max(feature_map,-1)
    features = np.max(features,-1)
    deeplearningfeatures = collections.OrderedDict()
    for ind_,f_ in enumerate(features):
    	deeplearningfeatures[str(ind_)] = f_
         
    return deeplearningfeatures


# salvataggio features in un file
featureDict = {}
for s in os.listdir(pathdicom):
    print(s)
    filename = os.path.join(pathdicom, s)

        
    for t in os.listdir(filename):

        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):

        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    
    deeplearningfeatures = featureextraction(readdatadicom,readdatanrrd) 

    result = deeplearningfeatures
    key = list(result.keys())
    key = key[0:]
        
    feature = []
    for jind in range(len(key)):
        feature.append(result[key[jind]])
        
    featureDict[s] = feature
    dictkey = key
    print(s)
    
dataframe = pd.DataFrame.from_dict(featureDict, orient='index', columns=dictkey)
dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\VGG19_NUOVO.csv')
    



100
(224, 224)
(224, 224, 1)
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
100
101
(224, 224)
(224, 224, 1)
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
101
102
(224, 224)
(224, 224, 1)
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
102
103
(224, 224)
(224, 224, 1)
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
103
104
(224, 224)
(224, 224, 1)
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
104
105
(224, 224)
(224, 224, 1)
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
105
106
(224, 224)
(224, 224, 1)
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
106
107
(224, 224)
(224, 224, 1)
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
107
108
(224, 224)
(224, 224, 1)
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
108
109
(224, 224)
(224, 224, 1)
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
109
110
(224, 224)
(224, 224, 1)
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
110
111
(224, 224)
(224, 

np.max(feature_map, -1): Si prende il massimo lungo l'ultima dimensione (lunghezza o altezza della mappa delle feature). Questo riduce la mappa delle feature da una matrice bidimensionale (per ogni canale) a un vettore monodimensionale, collassando una dimensione spaziale (ad esempio, comprimendo l'informazione di larghezza o altezza).
np.max(features, -1): Si ripete l'operazione di massimo su un'altra dimensione spaziale, riducendo ulteriormente il vettore monodimensionale a uno scalare per ogni canale. In pratica, per ogni canale si ottiene il valore massimo su tutta l'area della mappa delle feature. Questo passaggio riduce completamente la mappa delle feature a un singolo valore per canale.

In [18]:
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

model = ResNet50(weights='imagenet', include_top=False)

pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

# funzione per creazione maschera
def maskcroppingbox(images_array):
    images_array_2 = np.argwhere(images_array)
    
    (zstart, ystart, xstart), (zstop, ystop, xstop) = images_array_2.min(axis=0), images_array_2.max(axis=0) + 1
    return (zstart, ystart, xstart), (zstop, ystop, xstop)
        
def featureextraction(image_array, mask_array, patient_id):
    # Ridimensionamento della ROI
    (zstart, ystart, xstart), (zstop, ystop, xstop) = maskcroppingbox(mask_array)
    roi_images = image_array[zstart-1:zstop+1, ystart:ystop, xstart:xstop].transpose((2, 1, 0))
    roi_images1 = zoom(roi_images, zoom=[224/roi_images.shape[0], 224/roi_images.shape[1], 1], order=3)
    roi_images2 = np.array(roi_images1, dtype=float)

    # Prepara tutte le slice come un batch
    batch_slices = []
    for slice_index in range(roi_images2.shape[2]):
        slice_image = roi_images2[:, :, slice_index]
        x = image.img_to_array(slice_image)
        x = np.repeat(x, 3, axis=-1)  # Ripeti il canale se è un'immagine a singolo canale
        batch_slices.append(x)

    # Converti tutte le slice in un batch di input per ResNet
    batch_slices = np.array(batch_slices)
    batch_slices = preprocess_input(batch_slices)

    # Estrazione feature map per tutto il batch
    base_model_pool_features = model.predict(batch_slices)

    # Inizializza lista per salvare tutte le features
    all_features = []

    # Estrai le feature map per ogni slice e salva con il numero del paziente e della slice
    for slice_index in range(base_model_pool_features.shape[0]):
        feature_map = base_model_pool_features[slice_index]

        # Trasposizione e riduzione delle features
        feature_map = feature_map.transpose((2, 1, 0))
        features = np.max(feature_map, -1)
        features = np.max(features, -1)

        # Aggiungi le feature con il numero della slice e del paziente
        feature_entry = {'Patient': patient_id, 'Slice': slice_index}
        for ind_, f_ in enumerate(features):
            feature_entry[f'Feature_{ind_}'] = f_

        all_features.append(feature_entry)

    return all_features


all_feature_dicts = []

for s in os.listdir(pathdicom):
    print(s)
    filename = os.path.join(pathdicom, s)

    for t in os.listdir(filename):
        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):
        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    # Estrai tutte le features per tutte le slice
    patient_features = featureextraction(readdatadicom, readdatanrrd, patient_id=s)

    # Aggiungi le feature di tutte le slice per questo paziente
    all_feature_dicts.extend(patient_features)

# Crea il DataFrame con le feature di tutte le slice e pazienti
dataframe = pd.DataFrame(all_feature_dicts)

# Salva il DataFrame in un file CSV
dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\RESNET50_ALL_SLICES.csv', index=False)
    



100
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step
101
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 415ms/step
102
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 696ms/step
103
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 510ms/step
104
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 510ms/step
105
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 383ms/step
106
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 970ms/step
107
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 474ms/step
108
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 795ms/step
109
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 465ms/step
110
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 591ms/step
111
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 836ms/step
112
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 499ms/step
113
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 819ms/step
114
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 361ms/step
115
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 475ms/step
116
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 464ms/step
117
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 212ms/step
118
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 492ms/step
119
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 358ms/step
12
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 316ms/step
120
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 687ms/step
123
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 890ms/step
124
2/2 ━━━━━━━